# Geometry optimization

We use DFTK and the [GeometryOptimization](https://github.com/JuliaMolSim/GeometryOptimization.jl/)
package to find the minimal-energy bond length of the $H_2$ molecule.
First we set up an appropriate `DFTKCalculator` (see AtomsCalculators integration),
for which we use the LDA model just like in the Tutorial for silicon.

In [1]:
using DFTK

calc = DFTKCalculator(;
    model_kwargs = (; functionals=LDA()),        # model_DFT keyword arguments
    basis_kwargs = (; kgrid=[1, 1, 1], Ecut=10)  # PlaneWaveBasis keyword arguments
)

DFTKCalculator(functionals=Xc(lda_x, lda_c_pw), Ecut=10, kgrid=[1, 1, 1])

Next we set up an initial hydrogen molecule within a box of vacuum.
We use the parameters of the
[equivalent tutorial from ABINIT](https://docs.abinit.org/tutorial/base1/),
that is a simulation box of 10 bohr times 10 bohr times 10 bohr and a
pseudodojo pseudopotential.

In [2]:
using LinearAlgebra
using PseudoPotentialData

r0 = 1.4   # Initial bond length in Bohr
a  = 10.0  # Box size in Bohr

lattice = a * I(3)
H = ElementPsp(:H, PseudoFamily("dojo.nc.sr.pbe.v0_4_1.oncvpsp3.standard.upf"))
atoms = [H, H]
positions = [zeros(3), lattice \ [r0, 0., 0.]]

h2_crude = periodic_system(lattice, atoms, positions)

FlexibleSystem(H₂, periodic = TTT):
    bounding_box      : [      10        0        0;
                                0       10        0;
                                0        0       10]u"a₀"

    Atom(H,  [       0,        0,        0]u"a₀")
    Atom(H,  [     1.4,        0,        0]u"a₀")

       .------------.  
      /|            |  
     / |            |  
    *  |            |  
    |  |            |  
    |  |            |  
    |  .------------.  
    | /            /   
    |/            /    
    H-H----------*     


Finally we call `minimize_energy!` to start the geometry optimisation.
We use `verbosity=2` to get some insight into the minimisation.
With `verbosity=1` only a summarising table would be printed and with
`verbosity=0` (default) the minimisation would be quiet.

In [3]:
using GeometryOptimization
results = minimize_energy!(h2_crude, calc; tol_forces=2e-6, verbosity=2)
nothing  # hide

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -1.098147347254                   -0.89   0.80    8.0    4.27s
  2   -1.100783197250       -2.58       -1.74   0.80    1.0    1.93s
  3   -1.100802481321       -4.71       -2.81   0.80    1.0   39.8ms
  4   -1.100803401815       -6.04       -3.62   0.80    1.0   17.2ms
  5   -1.100803450682       -7.31       -4.18   0.80    2.0   24.7ms
  6   -1.100803451409       -9.14       -4.50   0.80    1.0   18.0ms
  7   -1.100803451661       -9.60       -5.86   0.80    1.0   20.2ms
  8   -1.100803451664      -11.51       -6.20   0.80    2.0   23.3ms
  9   -1.100803451664      -13.01       -6.65   0.80    1.0   25.0ms
 10   -1.100803451664      -13.78       -7.64   0.80    1.0   20.9ms
 11   -1.100803451664   +    -Inf       -7.61   0.80    2.0   23.2ms
 12   -1.100803451664   +  -15.05       -8.08   0.80    1.0   22.6ms
 13   -1.100803451664   +  -15.35  

Structure after optimisation (note that the atom has wrapped around)

In [4]:
results.system

FlexibleSystem(H₂, periodic = TTT):
    bounding_box      : [      10        0        0;
                                0       10        0;
                                0        0       10]u"a₀"

    Atom(H,  [-0.0640966, -7.09872e-13, -8.66745e-12]u"a₀")
    Atom(H,  [  1.4641, -3.79466e-12, -6.33527e-12]u"a₀")

       .-H----------H  
      /|            |  
     / |            |  
    *  |            |  
    |  |            |  
    |  |            |  
    |  .------------.  
    | /            /   
    |/            /    
    *------------*     


Compute final bond length:

In [5]:
rmin = norm(position(results.system[1]) - position(results.system[2]))
println("Optimal bond length: ", rmin)

Optimal bond length: 1.5281933000251298 a₀


Our results (1.486 Bohr) agrees with the
[equivalent tutorial from ABINIT](https://docs.abinit.org/tutorial/base1/).